
## Import The required Libraries
---



In [1]:
%%capture
!pip install --user "langchain"
!pip install --user "langchain-openai"
!pip install --user "huggingface"
!pip install --user "huggingface-hub"
!pip install --user "sentence-transformers"
!pip install --user "chromadb"
!pip install --user "wget == 3.2"

In [2]:
%%capture
!pip install --user "openai"
!pip install --user "langchain-community"

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import wget

In [2]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

file downloaded


## Read the document and print

In [10]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

## Text Splitting into Chunks

In [11]:
# Document loader loads the Documents
loader = TextLoader(filename)
documents = loader.load()

# Text Splitter Splits the Document into Text Chunks
text_splitter = CharacterTextSplitter(chunk_size=900, chunk_overlap=0)
text_chunks = text_splitter.split_documents(documents)
print(len(text_chunks)) # Prints the num of chucnks
print(text_chunks)

12
[Document(metadata={'source': 'companyPolicies.txt'}, page_content='1.\tCode of Conduct'), Document(metadata={'source': 'companyPolicies.txt'}, page_content="Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.\nIntegrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.\nRespect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.\nAccountability: We take responsibility for our actions and dec

From the ouput of print, you see that the document has been split into 16 chunks

## Embeddings and indexing
Creating Word Vector High Dimentional Vector Representations and using the Vector Database to Store the Vectors for Indexing

In [12]:
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(text_chunks, embeddings)  # store the embedding in docsearch using Chromadb
print('documents indexed')

documents indexed


This section Concludes creation of Question and Context Encoders Output and Vector Databases have been created.

# LLM MODEL

In [30]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 17.0 MB/s eta 0:00:00


Using a Free OpenAi GPT 2 Model, Can replace this with open AI Models

In [41]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

# Load GPT-2 model
model_id = "openai-community/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Create a text-generation pipeline
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)

# Wrap it in LangChain
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# Test it
print(llm("What is LangChain?"))


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is LangChain?

LangChain is a network application framework to run native-purpose code across various domains including web development, database development, network services, security and enterprise software.

LangChain provides both support for enterprise and mobile applications and has been designed for enterprise and mobile development and monitoring.

Why does LangChain need to work with enterprise tools?

LangChain includes tools on the enterprise software platform so that a company can easily and quickly manage multiple projects through the development process at


## Integrating the LLM with LangChain Retriver

In [42]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'query': 'Can I eat in company vehicles?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nPolicy Purpose: The Smoking Policy has been established to provide clear guidance and expectations concerning smoking on company premises. This policy is in place to ensure a safe and healthy environment for all employees, visitors, and the general public.\nDesignated Smoking Areas: Smoking is only permitted in designated smoking areas, as marked by appropriate signage. These areas have been chosen to minimize exposure to secondhand smoke and to maintain the overall cleanliness of the premises.\nSmoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed spaces is strictly prohibited. This includes electronic cigarettes and vaping devices.\nCompliance with Applicable Laws: All employees and visitors must adhere to relevant federa

## Create a Prompt Template to instruct the model to use the document context

In [43]:
prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [46]:
def qa():
    memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                               chain_type="stuff",
                                               retriever=docsearch.as_retriever(),
                                               memory = memory,
                                               get_chat_history=lambda h : h,
                                               return_source_documents=False)
    history = []
    while True:
        query = input("Question: ")

        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break

        result = qa({"question": query}, {"chat_history": history})

        history.append((query, result["answer"]))

        print("Answer: ", result["answer"])

In [48]:
qa()


Question: can i use mobile phone ?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance.
Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations.
Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device.

5.	Smoking Policy

The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of 

KeyboardInterrupt: Interrupted by user